# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [6]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [7]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [8]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
# import seaborn as sns
import colorsys

In [9]:
from sklearn.metrics import confusion_matrix

In [10]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [11]:
import utilities.helper
import trodes.read_exported

In [12]:
FONTSIZE = 20

In [13]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

## Inputs & Data

Explanation of each input and where it comes from.

In [14]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc/" # where data is saved should always be shown in the inputs
os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_3_long_comp"

In [15]:
LFP_SPECTRAL_DF = pd.read_pickle("./proc/{}_02_full_spectral.pkl".format(OUTPUT_PREFIX))


# LFP_SPECTRAL_DF = pd.read_pickle("./proc/rce_pilot_2_02_full_spectral_copy.pkl")
# LFP_SPECTRAL_DF = pd.read_pickle("./proc/rce_pilot_2_02_full_spectral.pkl")

In [16]:
# CHANNEL_MAP_AND_ALL_TRIALS_DF = pd.read_pickle("./proc/full_baseline_and_trial_lfp_traces.pkl")
# MERGED_TRIALS_AND_VIDEO = pd.read_pickle("./proc/trial_SLEAP_and_metadata.pkl")

In [17]:
theta_band = (4, 12)
gamma_band = (30, 50)
filter_band = (0, 56)

In [18]:
FONTSIZE=20

In [19]:
FULL_LFP_TRACES_PKL = "{}_03_spectral_bands.pkl".format(OUTPUT_PREFIX)

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Functions 

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

# Dropping unused columns

In [20]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

# Ideally functions are defined here first and then data is processed using the functions

# function names are short and in snake case all lowercase
# a function name should be unique but does not have to describe the function
# doc strings describe functions not function names




In [21]:
list(LFP_SPECTRAL_DF.columns)

['cohort',
 'session_dir',
 'tone_frames',
 'box_1_port_entry_frames',
 'box_2_port_entry_frames',
 'video_name',
 'notes',
 'session_path',
 'recording',
 'current_subject',
 'subject',
 'all_subjects',
 'first_timestamp',
 'last_timestamp',
 'video_timestamps',
 'tone_timestamps',
 'box_1_port_entry_timestamps',
 'box_2_port_entry_timestamps',
 'lfp_timestamps',
 'vHPC_lfp_trace',
 'BLA_lfp_trace',
 'LH_lfp_trace',
 'MD_lfp_trace',
 'mPFC_lfp_trace',
 'vHPC_lfp_RMS_filtered',
 'BLA_lfp_RMS_filtered',
 'LH_lfp_RMS_filtered',
 'MD_lfp_RMS_filtered',
 'mPFC_lfp_RMS_filtered',
 'vHPC_theta_band',
 'BLA_theta_band',
 'LH_theta_band',
 'MD_theta_band',
 'mPFC_theta_band',
 'vHPC_gamma_band',
 'BLA_gamma_band',
 'LH_gamma_band',
 'MD_gamma_band',
 'mPFC_gamma_band',
 'vHPC_theta_phase',
 'BLA_theta_phase',
 'LH_theta_phase',
 'MD_theta_phase',
 'mPFC_theta_phase',
 'vHPC_gamma_phase',
 'BLA_gamma_phase',
 'LH_gamma_phase',
 'MD_gamma_phase',
 'mPFC_gamma_phase',
 'vHPC_power_all_frequencies

In [22]:
dropping_columns = [col for col in LFP_SPECTRAL_DF if ("MAD" in col or "modified_zscore" in col or "RMS" in col or "mask" in col or "trace_filtered" in col) and "power" not in col]




In [23]:
dropping_columns

['vHPC_lfp_RMS_filtered',
 'BLA_lfp_RMS_filtered',
 'LH_lfp_RMS_filtered',
 'MD_lfp_RMS_filtered',
 'mPFC_lfp_RMS_filtered']

In [24]:
LFP_SPECTRAL_DF = LFP_SPECTRAL_DF.drop(columns=dropping_columns, errors="ignore")

## Filtering frequencies

In [25]:
list(LFP_SPECTRAL_DF.columns)

['cohort',
 'session_dir',
 'tone_frames',
 'box_1_port_entry_frames',
 'box_2_port_entry_frames',
 'video_name',
 'notes',
 'session_path',
 'recording',
 'current_subject',
 'subject',
 'all_subjects',
 'first_timestamp',
 'last_timestamp',
 'video_timestamps',
 'tone_timestamps',
 'box_1_port_entry_timestamps',
 'box_2_port_entry_timestamps',
 'lfp_timestamps',
 'vHPC_lfp_trace',
 'BLA_lfp_trace',
 'LH_lfp_trace',
 'MD_lfp_trace',
 'mPFC_lfp_trace',
 'vHPC_theta_band',
 'BLA_theta_band',
 'LH_theta_band',
 'MD_theta_band',
 'mPFC_theta_band',
 'vHPC_gamma_band',
 'BLA_gamma_band',
 'LH_gamma_band',
 'MD_gamma_band',
 'mPFC_gamma_band',
 'vHPC_theta_phase',
 'BLA_theta_phase',
 'LH_theta_phase',
 'MD_theta_phase',
 'mPFC_theta_phase',
 'vHPC_gamma_phase',
 'BLA_gamma_phase',
 'LH_gamma_phase',
 'MD_gamma_phase',
 'mPFC_gamma_phase',
 'vHPC_power_all_frequencies_all_windows',
 'BLA_power_all_frequencies_all_windows',
 'LH_power_all_frequencies_all_windows',
 'MD_power_all_frequencies_

In [26]:
all_frequencies_columns = [col for col in LFP_SPECTRAL_DF if "all_frequencies_all_windows" in col] 

In [27]:
for col in all_frequencies_columns:
    if "power" in col:
        LFP_SPECTRAL_DF[col] = LFP_SPECTRAL_DF.apply(lambda x: x[col][:, (x["power_calculation_frequencies"] >= filter_band[0]) & (x["power_calculation_frequencies"] < filter_band[1])], axis=1)
LFP_SPECTRAL_DF["power_calculation_frequencies"] = LFP_SPECTRAL_DF["power_calculation_frequencies"].apply(lambda x: x[filter_band[0]: filter_band[1]])


In [28]:
for col in all_frequencies_columns:
    if "coherence" in col:
        LFP_SPECTRAL_DF[col] = LFP_SPECTRAL_DF.apply(lambda x: x[col][:, (x["coherence_calculation_frequencies"] >= filter_band[0]) & (x["coherence_calculation_frequencies"] < filter_band[1])], axis=1)
LFP_SPECTRAL_DF["coherence_calculation_frequencies"] = LFP_SPECTRAL_DF["coherence_calculation_frequencies"].apply(lambda x: x[filter_band[0]: filter_band[1]])


In [29]:
for col in all_frequencies_columns:
    if "granger" in col:
        LFP_SPECTRAL_DF[col] = LFP_SPECTRAL_DF.apply(lambda x: x[col][:, (x["granger_calculation_frequencies"] >= filter_band[0]) & (x["granger_calculation_frequencies"] < filter_band[1])], axis=1)
LFP_SPECTRAL_DF["granger_calculation_frequencies"] = LFP_SPECTRAL_DF["granger_calculation_frequencies"].apply(lambda x: x[filter_band[0]: filter_band[1]])


In [30]:
for col in all_frequencies_columns:
    print(LFP_SPECTRAL_DF[col].iloc[0].shape)

(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)
(4581, 56)


In [31]:
LFP_SPECTRAL_DF["power_calculation_frequencies"].iloc[0].shape

(56,)

In [32]:
LFP_SPECTRAL_DF["coherence_calculation_frequencies"].iloc[0].shape

(56,)

In [33]:
LFP_SPECTRAL_DF["granger_calculation_frequencies"].iloc[0].shape

(56,)

## Filtering by theta and gamma band

- Filtering theta/gamma for coherence

In [34]:
coherence_columns = [col for col in LFP_SPECTRAL_DF.columns if "coherence" in col and "calculation" not in col and "time" not in col]

In [35]:
coherence_columns

['BLA_LH_coherence_all_frequencies_all_windows',
 'BLA_MD_coherence_all_frequencies_all_windows',
 'BLA_mPFC_coherence_all_frequencies_all_windows',
 'BLA_vHPC_coherence_all_frequencies_all_windows',
 'LH_MD_coherence_all_frequencies_all_windows',
 'LH_mPFC_coherence_all_frequencies_all_windows',
 'LH_vHPC_coherence_all_frequencies_all_windows',
 'MD_mPFC_coherence_all_frequencies_all_windows',
 'MD_vHPC_coherence_all_frequencies_all_windows',
 'mPFC_vHPC_coherence_all_frequencies_all_windows']

In [36]:
for col in coherence_columns:
    brain_region_name = "_".join(col.split("_")[:2])
    print(brain_region_name)
    theta_coherence_col = "{}_coherence_theta".format(brain_region_name)
    gamma_coherence_col = "{}_coherence_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_coherence_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["coherence_calculation_frequencies"] >= theta_band[0]) & (x["coherence_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_coherence_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["coherence_calculation_frequencies"] >= gamma_band[0]) & (x["coherence_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

BLA_LH
BLA_MD
BLA_mPFC
BLA_vHPC
LH_MD
LH_mPFC
LH_vHPC
MD_mPFC
MD_vHPC
mPFC_vHPC


In [37]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject,...,LH_mPFC_coherence_theta,LH_mPFC_coherence_gamma,LH_vHPC_coherence_theta,LH_vHPC_coherence_gamma,MD_mPFC_coherence_theta,MD_mPFC_coherence_gamma,MD_vHPC_coherence_theta,MD_vHPC_coherence_gamma,mPFC_vHPC_coherence_theta,mPFC_vHPC_coherence_gamma
0,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[1, 88], [1286, 1486], [3282, 3482], [4480, 4...","[[1, 88], [259, 278], [310, 365], [366, 382], ...",[],20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-1_t6b6_merged,3.1,...,"[0.6199009, nan, nan, nan, nan, nan, 0.7190465...","[0.4824039, nan, nan, nan, nan, nan, 0.3201964...","[0.24118073, nan, nan, nan, nan, nan, 0.509870...","[0.35312024, nan, nan, nan, nan, nan, 0.423012...","[0.70054024, nan, nan, 0.7242584, 0.68935406, ...","[0.4300425, nan, nan, 0.57167405, 0.50112396, ...","[0.3609763, nan, nan, 0.50976896, 0.50285643, ...","[0.15037705, nan, nan, 0.3243009, 0.3042064, 0...","[0.4029725, 0.16296464, 0.4309047, 0.40245545,...","[0.31822205, 0.39309344, 0.37264958, 0.2815909..."
1,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[2, 132], [1922, 2220], [4906, 5203], [6696, ...","[[2, 132], [387, 415], [463, 546], [548, 571],...",[],20240317_151922_long_comp_subj_3-1_and_3-3.2.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-1_t6b6_merged,3.1,...,"[0.6199009, nan, nan, nan, nan, nan, 0.7190465...","[0.4824039, nan, nan, nan, nan, nan, 0.3201964...","[0.24118073, nan, nan, nan, nan, nan, 0.509870...","[0.35312024, nan, nan, nan, nan, nan, 0.423012...","[0.70054024, nan, nan, 0.7242584, 0.68935406, ...","[0.4300425, nan, nan, 0.57167405, 0.50112396, ...","[0.3609763, nan, nan, 0.50976896, 0.50285643, ...","[0.15037705, nan, nan, 0.3243009, 0.3042064, 0...","[0.4029725, 0.16296464, 0.4309047, 0.40245545,...","[0.31822205, 0.39309344, 0.37264958, 0.2815909..."
2,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[1, 88], [1286, 1486], [3282, 3482], [4480, 4...","[[1, 88], [259, 278], [310, 365], [366, 382], ...",[],20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-3_t5b5_merged,3.3,...,"[0.82404065, 0.8251563, 0.8670573, 0.8001956, ...","[0.69295216, 0.6436656, 0.53512263, 0.44921312...","[0.73253983, 0.82799184, 0.7557261, 0.353088, ...","[0.6078736, 0.54056174, 0.5845927, 0.41842666,...","[0.8340234, 0.9002462, 0.8425386, 0.69482654, ...","[0.6832412, 0.59949577, 0.5694806, 0.4314444, ...","[0.6932799, 0.77787244, 0.70582354, 0.37044504...","[0.5074058, 0.5024076, 0.5942596, 0.5443486, 0...","[0.6331956, 0.7959172, 0.73796403, 0.5448313, ...","[0.47152033, 0.5181883, 0.49669346, 0.44575334..."
3,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[2, 132], [1922, 2220], [4906, 5203], [6696, ...","[[2, 132], [387, 415], [463, 546], [548, 571],...",[],20240317_151922_long_comp_subj_3-1_and_3-3.2.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-3_t5b5_merged,3.3,...,"[0.82404065, 0.8251563, 0.8670573, 0.8001956, ...","[0.69295216, 0.6436656, 0.53512263, 0.44921312...","[0.73253983, 0.82799184, 0.7557261, 0.353088, ...","[0.6078736, 0.54056174, 0.5845927, 0.41842666,...","[0.8340234, 0.9002462, 0.8425386, 0.69482654, ...","[0.6832412, 0.59949577, 0.5694806, 0.4314444, ...","[0.6932799, 0.77787244, 0.70582354, 0.37044504...","[0.5074058, 0.5024076, 0.5942596, 0.5443486, 0...","[0.6331956, 0.7959172, 0.73796403, 0.5448313, ...","[0.47152033, 0.5181883, 0.49669346, 0.44575334..."
4,rce_pilot_3,20240317_172017_long_comp_subj_4-2_and_4-3,"[[1, 102], [1300, 1501], [3297, 3497], [4494, ...","[[1, 102], [115, 153], [191, 245], [261, 263],...",[],20240317_172017_long_comp_subj_4-2_and_4-3.1.v...,NaN,/scratch/back_up/reward_co

In [38]:
for col in LFP_SPECTRAL_DF.columns:
    print(col)

cohort
session_dir
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
video_name
notes
session_path
recording
current_subject
subject
all_subjects
first_timestamp
last_timestamp
video_timestamps
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
vHPC_lfp_trace
BLA_lfp_trace
LH_lfp_trace
MD_lfp_trace
mPFC_lfp_trace
vHPC_theta_band
BLA_theta_band
LH_theta_band
MD_theta_band
mPFC_theta_band
vHPC_gamma_band
BLA_gamma_band
LH_gamma_band
MD_gamma_band
mPFC_gamma_band
vHPC_theta_phase
BLA_theta_phase
LH_theta_phase
MD_theta_phase
mPFC_theta_phase
vHPC_gamma_phase
BLA_gamma_phase
LH_gamma_phase
MD_gamma_phase
mPFC_gamma_phase
vHPC_power_all_frequencies_all_windows
BLA_power_all_frequencies_all_windows
LH_power_all_frequencies_all_windows
MD_power_all_frequencies_all_windows
mPFC_power_all_frequencies_all_windows
power_timestamps
power_calculation_frequencies
BLA_LH_coherence_all_frequencies_all_windows
BLA_MD_coherence_all_frequencies_all_windows
BL

- Filtering theta/gamma for granger

In [39]:
granger_columns = [col for col in LFP_SPECTRAL_DF.columns if "granger" in col and "calculation" not in col and "time" not in col]

In [40]:
granger_columns

['BLA_LH_granger_all_frequencies_all_windows',
 'LH_BLA_granger_all_frequencies_all_windows',
 'BLA_MD_granger_all_frequencies_all_windows',
 'MD_BLA_granger_all_frequencies_all_windows',
 'BLA_mPFC_granger_all_frequencies_all_windows',
 'mPFC_BLA_granger_all_frequencies_all_windows',
 'BLA_vHPC_granger_all_frequencies_all_windows',
 'vHPC_BLA_granger_all_frequencies_all_windows',
 'LH_MD_granger_all_frequencies_all_windows',
 'MD_LH_granger_all_frequencies_all_windows',
 'LH_mPFC_granger_all_frequencies_all_windows',
 'mPFC_LH_granger_all_frequencies_all_windows',
 'LH_vHPC_granger_all_frequencies_all_windows',
 'vHPC_LH_granger_all_frequencies_all_windows',
 'MD_mPFC_granger_all_frequencies_all_windows',
 'mPFC_MD_granger_all_frequencies_all_windows',
 'MD_vHPC_granger_all_frequencies_all_windows',
 'vHPC_MD_granger_all_frequencies_all_windows',
 'mPFC_vHPC_granger_all_frequencies_all_windows',
 'vHPC_mPFC_granger_all_frequencies_all_windows']

In [41]:
for col in granger_columns:
    brain_region_name = "-to-".join(col.split("_")[:2])
    print(brain_region_name)
    theta_granger_col = "{}_granger_theta".format(brain_region_name)
    gamma_granger_col = "{}_granger_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_granger_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["granger_calculation_frequencies"] >= theta_band[0]) & (x["granger_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_granger_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["granger_calculation_frequencies"] >= gamma_band[0]) & (x["granger_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

BLA-to-LH
LH-to-BLA
BLA-to-MD
MD-to-BLA
BLA-to-mPFC
mPFC-to-BLA
BLA-to-vHPC
vHPC-to-BLA
LH-to-MD
MD-to-LH
LH-to-mPFC
mPFC-to-LH
LH-to-vHPC
vHPC-to-LH
MD-to-mPFC
mPFC-to-MD
MD-to-vHPC
vHPC-to-MD
mPFC-to-vHPC
vHPC-to-mPFC


In [42]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject,...,mPFC-to-MD_granger_theta,mPFC-to-MD_granger_gamma,MD-to-vHPC_granger_theta,MD-to-vHPC_granger_gamma,vHPC-to-MD_granger_theta,vHPC-to-MD_granger_gamma,mPFC-to-vHPC_granger_theta,mPFC-to-vHPC_granger_gamma,vHPC-to-mPFC_granger_theta,vHPC-to-mPFC_granger_gamma
0,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[1, 88], [1286, 1486], [3282, 3482], [4480, 4...","[[1, 88], [259, 278], [310, 365], [366, 382], ...",[],20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-1_t6b6_merged,3.1,...,"[0.55530393, nan, nan, 1.144514, 0.4849566, 1....","[0.43555573, nan, nan, 0.4677967, 0.28638032, ...","[0.21118473, nan, nan, 0.5874789, 0.5205787, 0...","[0.09165167, nan, nan, 0.43196425, 0.33087164,...","[0.4447786, nan, nan, 0.19807723, 0.29667133, ...","[0.15449163, nan, nan, 0.17275214, 0.29230323,...","[0.26188377, 0.28431654, 0.87577665, 0.5741280...","[0.32400984, 0.4826204, 0.49196243, 0.6095017,...","[0.47272158, 0.21452194, 0.193596, 0.3200296, ...","[0.21859355, 0.3514257, 0.40666527, 0.169144, ..."
1,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[2, 132], [1922, 2220], [4906, 5203], [6696, ...","[[2, 132], [387, 415], [463, 546], [548, 571],...",[],20240317_151922_long_comp_subj_3-1_and_3-3.2.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-1_t6b6_merged,3.1,...,"[0.55530393, nan, nan, 1.144514, 0.4849566, 1....","[0.43555573, nan, nan, 0.4677967, 0.28638032, ...","[0.21118473, nan, nan, 0.5874789, 0.5205787, 0...","[0.09165167, nan, nan, 0.43196425, 0.33087164,...","[0.4447786, nan, nan, 0.19807723, 0.29667133, ...","[0.15449163, nan, nan, 0.17275214, 0.29230323,...","[0.26188377, 0.28431654, 0.87577665, 0.5741280...","[0.32400984, 0.4826204, 0.49196243, 0.6095017,...","[0.47272158, 0.21452194, 0.193596, 0.3200296, ...","[0.21859355, 0.3514257, 0.40666527, 0.169144, ..."
2,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[1, 88], [1286, 1486], [3282, 3482], [4480, 4...","[[1, 88], [259, 278], [310, 365], [366, 382], ...",[],20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-3_t5b5_merged,3.3,...,"[0.29298076, 0.34389672, 0.10124099, 0.3364377...","[0.23826034, 0.31733543, 0.1894333, 0.23274863...","[0.6946653, 0.99855816, 0.8788032, 0.25746834,...","[0.42479968, 0.17542012, 0.31873545, 0.4310779...","[0.53250146, 0.65065455, 0.672333, 0.37134326,...","[0.2533988, 0.35809487, 0.45942515, 0.3916846,...","[0.700121, 0.6060421, 1.0413619, 0.55516905, 0...","[0.17148598, 0.55180275, 0.5668941, 0.45296097...","[0.22400546, 0.48265776, 0.40391633, 0.5957653...","[0.18092635, 0.2230971, 0.25372884, 0.28041154..."
3,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[2, 132], [1922, 2220], [4906, 5203], [6696, ...","[[2, 132], [387, 415], [463, 546], [548, 571],...",[],20240317_151922_long_comp_subj_3-1_and_3-3.2.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-3_t5b5_merged,3.3,...,"[0.29298076, 0.34389672, 0.10124099, 0.3364377...","[0.23826034, 0.31733543, 0.1894333, 0.23274863...","[0.6946653, 0.99855816, 0.8788032, 0.25746834,...","[0.42479968, 0.17542012, 0.31873545, 0.4310779...","[0.53250146, 0.65065455, 0.672333, 0.37134326,...","[0.2533988, 0.35809487, 0.45942515, 0.3916846,...","[0.700121, 0.6060421, 1.0413619, 0.55516905, 0...","[0.17148598, 0.55180275, 0.5668941, 0.45296097...","[0.22400546, 0.48265776, 0.40391633, 0.5957653...","[0.18092635, 0.2230971, 0.25372884, 0.28041154..."
4,rce_pilot_3,20240317_172017_long_comp_subj_4-2_and_4-3,"[[1, 102], [1300, 1501], [3297, 3497], [4494, ...","[[1, 102], [115, 153], [191, 245], [261, 263],...",[],20240317_172017_long_comp_subj_4-2_and_4-3.1.v...,NaN,/scratch/bac

- Filtering theta/gamma for power

In [43]:
power_columns = [col for col in LFP_SPECTRAL_DF.columns if "power" in col and "calculation" not in col and "time" not in col]

In [44]:
power_columns

['vHPC_power_all_frequencies_all_windows',
 'BLA_power_all_frequencies_all_windows',
 'LH_power_all_frequencies_all_windows',
 'MD_power_all_frequencies_all_windows',
 'mPFC_power_all_frequencies_all_windows']

In [45]:
LFP_SPECTRAL_DF["power_calculation_frequencies"].apply(lambda x: (x >= theta_band[0]) & (x <= theta_band[1])).iloc[0]

array([False, False, False, False,  True,  True,  True,  True,  True,
        True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [46]:
for col in power_columns:
    # brain_region_name = col.split("_")[0]
    brain_region_name = col.split("power")[0]
    brain_region_name = brain_region_name.strip("_")
    print(brain_region_name)
    theta_power_col = "{}_power_theta".format(brain_region_name)
    gamma_power_col = "{}_power_gamma".format(brain_region_name)
    LFP_SPECTRAL_DF["{}".format(theta_power_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["power_calculation_frequencies"] >= theta_band[0]) & (x["power_calculation_frequencies"] <= theta_band[1])], axis=1), axis=1)
    LFP_SPECTRAL_DF["{}".format(gamma_power_col)] = LFP_SPECTRAL_DF.apply(lambda x: np.nanmean(x[col][:, (x["power_calculation_frequencies"] >= gamma_band[0]) & (x["power_calculation_frequencies"] <= gamma_band[1])], axis=1), axis=1)

vHPC
BLA
LH
MD
mPFC


In [47]:
LFP_SPECTRAL_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,notes,session_path,recording,current_subject,...,vHPC_power_theta,vHPC_power_gamma,BLA_power_theta,BLA_power_gamma,LH_power_theta,LH_power_gamma,MD_power_theta,MD_power_gamma,mPFC_power_theta,mPFC_power_gamma
0,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[1, 88], [1286, 1486], [3282, 3482], [4480, 4...","[[1, 88], [259, 278], [310, 365], [366, 382], ...",[],20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-1_t6b6_merged,3.1,...,"[0.029989537, 0.05074093, 0.045202438, 0.03654...","[0.0021393485, 0.001990721, 0.0016591972, 0.00...","[0.01576374, 0.017160619, nan, nan, 0.01513487...","[0.0029858148, 0.0055226046, nan, nan, 0.00234...","[0.014374395, nan, nan, nan, nan, nan, 0.02792...","[0.0015033478, nan, nan, nan, nan, nan, 0.0011...","[0.015749235, nan, nan, 0.013588659, 0.0394984...","[0.0012602397, nan, nan, 0.0016373609, 0.00110...","[0.02549853, 0.01599767, 0.019950056, 0.041470...","[0.0018852353, 0.0019809369, 0.0013036462, 0.0..."
1,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[2, 132], [1922, 2220], [4906, 5203], [6696, ...","[[2, 132], [387, 415], [463, 546], [548, 571],...",[],20240317_151922_long_comp_subj_3-1_and_3-3.2.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-1_t6b6_merged,3.1,...,"[0.029989537, 0.05074093, 0.045202438, 0.03654...","[0.0021393485, 0.001990721, 0.0016591972, 0.00...","[0.01576374, 0.017160619, nan, nan, 0.01513487...","[0.0029858148, 0.0055226046, nan, nan, 0.00234...","[0.014374395, nan, nan, nan, nan, nan, 0.02792...","[0.0015033478, nan, nan, nan, nan, nan, 0.0011...","[0.015749235, nan, nan, 0.013588659, 0.0394984...","[0.0012602397, nan, nan, 0.0016373609, 0.00110...","[0.02549853, 0.01599767, 0.019950056, 0.041470...","[0.0018852353, 0.0019809369, 0.0013036462, 0.0..."
2,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[1, 88], [1286, 1486], [3282, 3482], [4480, 4...","[[1, 88], [259, 278], [310, 365], [366, 382], ...",[],20240317_151922_long_comp_subj_3-1_and_3-3.1.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-3_t5b5_merged,3.3,...,"[0.030139383, 0.038337562, 0.035997633, 0.0163...","[0.0013218134, 0.001314961, 0.0014802001, 0.00...","[0.035382226, 0.031998232, 0.029764285, 0.0189...","[0.0013072899, 0.0012992765, 0.0010765253, 0.0...","[0.03857511, 0.028973801, 0.029779362, 0.02279...","[0.0009867165, 0.0008933175, 0.00090869045, 0....","[0.01698081, 0.014525875, 0.015493691, 0.01093...","[0.0012375844, 0.0010923791, 0.0008535082, 0.0...","[0.020940535, 0.027591836, 0.029557036, 0.0211...","[0.0014317904, 0.0010592391, 0.0012028972, 0.0..."
3,rce_pilot_3,20240317_151922_long_comp_subj_3-1_and_3-3,"[[2, 132], [1922, 2220], [4906, 5203], [6696, ...","[[2, 132], [387, 415], [463, 546], [548, 571],...",[],20240317_151922_long_comp_subj_3-1_and_3-3.2.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_151922_long_comp_subj_3-3_t5b5_merged,3.3,...,"[0.030139383, 0.038337562, 0.035997633, 0.0163...","[0.0013218134, 0.001314961, 0.0014802001, 0.00...","[0.035382226, 0.031998232, 0.029764285, 0.0189...","[0.0013072899, 0.0012992765, 0.0010765253, 0.0...","[0.03857511, 0.028973801, 0.029779362, 0.02279...","[0.0009867165, 0.0008933175, 0.00090869045, 0....","[0.01698081, 0.014525875, 0.015493691, 0.01093...","[0.0012375844, 0.0010923791, 0.0008535082, 0.0...","[0.020940535, 0.027591836, 0.029557036, 0.0211...","[0.0014317904, 0.0010592391, 0.0012028972, 0.0..."
4,rce_pilot_3,20240317_172017_long_comp_subj_4-2_and_4-3,"[[1, 102], [1300, 1501], [3297, 3497], [4494, ...","[[1, 102], [115, 153], [191, 245], [261, 263],...",[],20240317_172017_long_comp_subj_4-2_and_4-3.1.v...,NaN,/scratch/back_up/reward_competition_extention/...,20240317_172017_long_comp_subj_4-2_t6b6_merged,4.2,...,"[0.0

In [48]:
LFP_SPECTRAL_DF.to_pickle(os.path.join(OUTPUT_DIR, FULL_LFP_TRACES_PKL))

In [49]:
for col in LFP_SPECTRAL_DF.columns:
    print(col)

cohort
session_dir
tone_frames
box_1_port_entry_frames
box_2_port_entry_frames
video_name
notes
session_path
recording
current_subject
subject
all_subjects
first_timestamp
last_timestamp
video_timestamps
tone_timestamps
box_1_port_entry_timestamps
box_2_port_entry_timestamps
lfp_timestamps
vHPC_lfp_trace
BLA_lfp_trace
LH_lfp_trace
MD_lfp_trace
mPFC_lfp_trace
vHPC_theta_band
BLA_theta_band
LH_theta_band
MD_theta_band
mPFC_theta_band
vHPC_gamma_band
BLA_gamma_band
LH_gamma_band
MD_gamma_band
mPFC_gamma_band
vHPC_theta_phase
BLA_theta_phase
LH_theta_phase
MD_theta_phase
mPFC_theta_phase
vHPC_gamma_phase
BLA_gamma_phase
LH_gamma_phase
MD_gamma_phase
mPFC_gamma_phase
vHPC_power_all_frequencies_all_windows
BLA_power_all_frequencies_all_windows
LH_power_all_frequencies_all_windows
MD_power_all_frequencies_all_windows
mPFC_power_all_frequencies_all_windows
power_timestamps
power_calculation_frequencies
BLA_LH_coherence_all_frequencies_all_windows
BLA_MD_coherence_all_frequencies_all_windows
BL

In [50]:
raise ValueError()

ValueError: 